# Inverter Grid Test

In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries
import matplotlib.pyplot as plt
import numpy as np
import dpsimpy

In [ ]:
time_step = 0.000001
final_time = 0.05
sim_name = 'DP_Inverter_Grid_Test'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

frequencies = [50, 19850, 19950, 20050, 20150, 39750, 39950, 40050, 40250]

n1 = dpsimpy.dp.SimNode('n1')
n2 = dpsimpy.dp.SimNode('n2')
n3 = dpsimpy.dp.SimNode('n3')
n4 = dpsimpy.dp.SimNode('n4')
n5 = dpsimpy.dp.SimNode('n5')

log_level = dpsimpy.LogLevel.info

inv = dpsimpy.dp.ph1.Inverter('inv', log_level)
inv.set_parameters(carrier_harms=[2,2,2,2,4,4,4,4], modul_harms=[-3,-1,1,3,-5,-1,1,5],
    input_voltage=360, ratio=0.87, phase=0)

r1 = dpsimpy.dp.ph1.Resistor('r1', log_level)
r1.set_parameters(0.1)
l1 = dpsimpy.dp.ph1.Inductor('l1', log_level)
l1.set_parameters(600e-6)
r2 = dpsimpy.dp.ph1.Resistor('r2', log_level)
r2.set_parameters(0.1 + 0.001)
l2 = dpsimpy.dp.ph1.Inductor('l2', log_level)
l2.set_parameters(150e-6 + 0.001/(2*np.pi*50))
c1 = dpsimpy.dp.ph1.Capacitor('c1', log_level)
c1.set_parameters(10e-6)
rc = dpsimpy.dp.ph1.Capacitor('rc', log_level)
rc.set_parameters(1e-6)
grid = dpsimpy.dp.ph1.VoltageSource('grid', log_level)
grid.set_parameters(complex(0, -311.1270))

inv.connect([n1])
r1.connect([n1, n2])
l1.connect([n2, n3])
c1.connect([dpsimpy.dp.SimNode.gnd, n3])
rc.connect([dpsimpy.dp.SimNode.gnd, n3])
r2.connect([n3, n4])
l2.connect([n4, n5])
grid.connect([dpsimpy.dp.SimNode.gnd, n5])

sys = dpsimpy.SystemTopology(50, frequencies, [n1, n2, n3, n4, n5],
    [inv, r1, l1, r2, l2, c1, rc, grid])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1, 1, 9)
logger.log_attribute('v2', 'v', n2, 1, 1)
logger.log_attribute('v3', 'v', n3, 1, 9)
logger.log_attribute('v4', 'v', n4, 1, 1)
logger.log_attribute('v5', 'v', n5, 1, 1)
logger.log_attribute('i12', 'i_intf', r1, 1, 1)
logger.log_attribute('i34', 'i_intf', r2, 1, 1)

sim = dpsimpy.Simulation(sim_name, log_level)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.add_logger(logger)
sim.do_frequency_parallelization(False)

sim.run()

In [ ]:
# read results
logs_dir = './logs/'
log_path =  logs_dir + 'DP_Inverter_Grid_Test/' + 'DP_Inverter_Grid_Test.csv'
print(log_path)

ts_dpsim = rt.read_timeseries_dpsim(log_path)

In [ ]:
ts_list_v1 = [
    ts_dpsim['v1_0_0'],
    ts_dpsim['v1_0_1'],
    ts_dpsim['v1_0_2'],
    ts_dpsim['v1_0_3'],
    ts_dpsim['v1_0_4'],
    ts_dpsim['v1_0_5'],
    ts_dpsim['v1_0_6'],
    ts_dpsim['v1_0_7'],
    ts_dpsim['v1_0_8']]

ts_list_v3 = [
    ts_dpsim['v3_0_0'],
    ts_dpsim['v3_0_1'],
    ts_dpsim['v3_0_2'],
    ts_dpsim['v3_0_3'],
    ts_dpsim['v3_0_4'],
    ts_dpsim['v3_0_5'],
    ts_dpsim['v3_0_6'],
    ts_dpsim['v3_0_7'],
    ts_dpsim['v3_0_8']]

freq_list = [ 50,
    19850, 19950, 20050, 20150,
    39750, 39950, 40050, 40250 ]

In [ ]:
ts_dpsim_emt_v1 = TimeSeries.create_emt_from_dp(ts_list_v1, freq_list)
plt.plot(ts_dpsim_emt_v1.time, ts_dpsim_emt_v1.values)
plt.xlim(0, 0.05)
plt.show()

In [ ]:
ts_dpsim_emt_v3 = TimeSeries.create_emt_from_dp(ts_list_v3, freq_list)
plt.plot(ts_dpsim_emt_v3.time, ts_dpsim_emt_v3.values)
plt.xlim(0, 0.05)
plt.show()

In [ ]:
ts_dpsim_emt_v5 = ts_dpsim['v5_0_0'].frequency_shift(50)

plt.plot(ts_dpsim_emt_v1.time, ts_dpsim_emt_v1.values)
plt.plot(ts_dpsim_emt_v3.time, ts_dpsim_emt_v3.values)
plt.plot(ts_dpsim_emt_v5.time, ts_dpsim_emt_v5.values, linestyle='--')
plt.xlim(0, 0.05)
plt.show()

## Validation against Matlab model

The Matlab model is considering 4 harmonics only. Therefore the DPsim results might be slightly different depending on the number of considered harmonics.

In [ ]:
import os
import urllib.request

if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://raw.githubusercontent.com/dpsim-simulator/reference-results/master/Simulink/Inverter/ML_SP_Inverter_OL_v_i.csv'
local_file = 'reference-results/ML_SP_Inverter_OL_v_i.csv'
urllib.request.urlretrieve(url, local_file) 

ts_ml_inverter = rt.read_timeseries_simulink(local_file)

In [ ]:
plt.plot(ts_ml_inverter['v_inv'].time, ts_ml_inverter['v_inv'].values)

#plt.figure(figsize=(20,10))
plt.figure()
plt.plot(ts_ml_inverter['v_c'].time, ts_ml_inverter['v_c'].values)
plt.plot(ts_dpsim_emt_v3.time, ts_dpsim_emt_v3.values)
plt.xlim(0.0245, 0.0255)
plt.ylim(308, 315)
plt.show()

In [ ]:
import numpy as np
diff = ts_ml_inverter['v_c'].values[:5000] - np.interp(ts_ml_inverter['v_c'].time[:5000], ts_dpsim_emt_v3.time, ts_dpsim_emt_v3.values)
print(max(diff))
assert np.all(diff) < 10

## Validation against Simulink switching model

In [ ]:
import os
import urllib.request

if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://raw.githubusercontent.com/dpsim-simulator/reference-results/master/Simulink/Inverter/SL_SP_Inverter_OL_v_i.csv'
local_file = 'reference-results/SL_SP_Inverter_OL_v_i.csv'
urllib.request.urlretrieve(url, local_file) 

ts_sl_inverter = rt.read_timeseries_simulink(local_file)

In [ ]:
plt.plot(ts_sl_inverter['v_inv'].time, ts_sl_inverter['v_inv'].values)
plt.show()

In [ ]:
plt.plot(ts_sl_inverter['v_c'].time, ts_sl_inverter['v_c'].values, label='EMT')
plt.plot(ts_dpsim_emt_v3.time, ts_dpsim_emt_v3.values, label='DP', linestyle='--')

ax = plt.gca()
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
plt.xlim(0.0445,0.0456)
plt.ylim(308,315)
plt.xlabel('time (s)')
plt.ylabel('voltage (V)')
plt.grid()
plt.legend(loc='upper right')
#ax = plt.gca()
#ax.xaxis.set_major_locator(plt.MaxNLocator(5))
plt.tight_layout()
plt.savefig('dp_simulink_inv_8_harm.pdf')
plt.savefig('dp_simulink_inv_8_harm.png', dpi=300)

In [ ]:
plt.plot(ts_sl_inverter['v_c'].time, ts_sl_inverter['v_c'].values, label='simulink')
plt.plot(ts_dpsim_emt_v3.time, ts_dpsim_emt_v3.values, linestyle=":", label='dyn. phasor')
plt.xlim(0, 0.05)

plt.xlabel('time (s)')
plt.ylabel('voltage (V)')
plt.grid()
plt.legend(loc='lower right')
plt.tight_layout()

plt.savefig('dp_simulink_inv_8_harm_complete.pdf')
plt.savefig('dp_simulink_inv_8_harm_complete.png', dpi=300)